# COMP551 Mini Project 2 - IMDB Sentiment Analysis  
This is the codes of mini project2 - IMDB Sentiment Analysis.  

## AUTHORS
Pengnan Fan, ID#260768510  

## TASKS
1. **Bernoulli Naive Bayes** (w/o any external library).  
2. **At least 2** out of 3 classifiers from the SciKit. i.e. suggestions: logistic regression, decision tree, or support vector machines  
3. **At least 2** different features extraction pipelines for processing the data.  
4. A model validation. i.e. **K-fold cross validation**  

## UPDATES
**<January 6, 2019>** Pengnan Fan create this notebook and implements functions readTrainData and readTestData  
> * ~~**readTrainData(address:String):DataFrame**~~  
> ~~This function takes a string **address** which indicates the address of your train data and will load comments and isPositive to a DataFrame.~~  
> * ~~**readTestData(address:String):DataFrame**~~  
> ~~This function takes a string **address** which indicates the address of your test data and will load comments and isPositive to a DataFrame. **Note: all isPositive is initialized as 0**~~  
> * **Learning set** 25000 in total
> * **Test set** 25000 in total  

**<January 7, 2019>** Pengnan Fan implements wordsFrequencyNaive and wordsFrequencyStopword
> * **wordsFrequencyNaive(dataSet:DataFrame)**  
> This function takes a DataFrame **dataSet** and calculate the naive word frequency  
> * **wordsFrequencyStopword(dataSet:DataFrame)**  
> This function takes a DataFrame **dataSet** and calculate the word frequency without stopwords  

**<January 9, 2019>** Pengnan Fan uses sklearn.datasets.load_files(address:str) to load all datas. It helps to increase the speed of loading. And the implementation of Bernoulli naive Bayes has been started.  
> * **sklearn.datasets.load_files** - [Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html)

**<January 10, 2019>** Pengnan Fan implements numOfExistanceNaive and numOfExistanceStopword
> * **numOfExistanceNaive(dataset:Bunch)**  
> This function takes a Bunch **dataSet** and calculate the naive existance of words~~  
> * **numOfExistanceStopword(dataset:Bunch)**  
> This funciton takes a Bunch **dataSet** and calculate the existance of words without stopwords  

**<January 11, 2019>** Pengnan Fan implements bernoulliNaiveBayes and evaluation. Also, the dataset is changed by using loadData  
> * ~~**bernoulliNaiveBayes(dataSet:Bunch, totalWordFreq:set, negWordFreq:set, posWordFreq:set, numOfExamples:list)**~~  
> ~~This function takes a Bunch **dataSet** as learning set and uses three sets of **totalWordFreq**, **negWordFreq**, **posWordFreq** and a list **numOfExamples** to calculates related probabilities. Note: It is not completely corrent. Fix later today.~~  
> ~~* **evaluation(dataSet:Bunch, prediction:list)**  
> This function takes a Bunch **dataSet** and a list **prediction** to generate a set containing true pos\true neg\false pos\false neg
> * **loadData(address:str)**~~  
> This function takes a string **address** and generate a dict of 3 lists (pos, neg, all) of dict ('comment', 'isPos')  

**<January 13, 2019>** Pengnan Fan fixs numOfExistanceStopword and bernoulliNaiveBayes
> * **advancedNumOfExistance(dataset:dict of list of dict)**  
> This funciton takes a dict of list of dict **dataSet** and calculate the existance of words without stopwords, puctuations, and duplicates.  
> * **bernoulliNaiveBayes(dataSet:list of dict, wordExistance:dict of list, size:dict of list)**  
> This function takes a list of dict **dataSet** as predicting set and uses a dict of list **wordExistance** and a dict of list **size** to calculates related probabilities.  
> * **evaluation(dataSet:list of dict, prediction:list)**  
> This function takes a list of dict **dataSet** and a list **prediction** to generate a set containing true pos\true neg\false pos\false neg  
> * **Outcome** TP =  11848, TN =  11482, FP =  1018, FN =  652  
>> accuracy =  93.32 %

In [1]:
import pandas as pd
import glob
import sys
import math
from time import sleep
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import nltk
import numpy
import scipy
import sklearn.datasets
import contractions
from itertools import groupby
import string

## Task 0 - Preprocessing Data

### Task 0.1 - Data Loading

In [24]:
# Task 0 - Preprocessing Data

# Please add your address here as string
ADDRESS_TRAIN_PENGNAN = "D:\\McGill\\19Fall\\COMP 551\\Projects\\Project2\\comp-551-imbd-sentiment-classification\\train"
ADDRESS_TEST_PENGNAN = "D:\\McGill\\19Fall\\COMP 551\\Projects\\Project2\\comp-551-imbd-sentiment-classification\\test"



# @author Pengnan Fan
# @param address of train set
# @return a dict of list of dict
def loadData(address):
    print("Start loading negative set")
    neg = sklearn.datasets.load_files(address, categories={"neg"})
    print("Complete loading negative set")
    print("Start loading positive set")
    pos = sklearn.datasets.load_files(address, categories={"pos"})
    print("Complete loading postive set")
    
    negSet = list()
    count = 0
    size = len(neg.data)
    for x in neg.data:
        negSet.append({"comment":x.decode('utf-8'), "isPos":0})
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(size)) * 100)
        sys.stdout.write("Preparing negative set: [%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    print()
    
    posSet = list()
    count = 0
    size = len(pos.data)
    print(size)
    for x in pos.data:
        posSet.append({"comment":x.decode('utf-8'), "isPos":1})
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(size)) * 100)
        sys.stdout.write("Prepare positive set: [%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    output = {'pos':posSet, 'neg':negSet, 'all':posSet+negSet}
    print("\nFinish preparing")
    return output

# Advanced loader
# @return: Bunch

trainSet = loadData(ADDRESS_TRAIN_PENGNAN)

Start loading negative set
Complete loading negative set
Start loading positive set
Complete loading postive set
Preparing negative set: [=================== ] 99%
12500
Prepare positive set: [=================== ] 99%
Finish preparing


In [26]:
# An example of the dataset
print(len(trainSet['neg']))

12500


### Task 0.2 - Naive Word Frequency

In [27]:
# @author Pengnan Fan
# @param dataSet: set of comments
# @return naiveCount: naive word frequency
def wordsFrequencyNaive(dataSet):
    naiveCount = dict()
    totalComments = []
    count = 0
    lenW = len(dataSet['pos'])
    
    print("Start counting naive word frequency of positive set")
    for comment in dataSet['pos']:
        totalComments+=comment['comment'].lower().split(" ")
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    print("\nComplete counting naive word frequency of positive set")
    
    posCount = Counter(x for x in totalComments)
    
    totalComments = []
    count = 0
    lenW = len(dataSet['neg'])
    
    print("Start counting naive word frequency of negative set")
    for comment in dataSet['neg']:
        totalComments+=comment['comment'].lower().split(" ")
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    print("\nComplete counting naive word frequency of negative set")
    
    negCount = Counter(x for x in totalComments)
    
    totalComments = []
    count = 0
    lenW = len(dataSet['all'])
    
    print("Start counting naive word frequency of all set")
    for comment in dataSet['all']:
        totalComments+=comment['comment'].lower().split(" ")
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    print("\nComplete counting naive word frequency of all set")
    
    allCount = Counter(x for x in totalComments)
    
    naiveCount = {'pos':posCount, 'neg':negCount, 'all':allCount}
    return naiveCount


# naiveFrequency = wordsFrequencyNaive(trainSet)

In [28]:
# An example of naive word frequency
'''
print(naiveFrequency.most_common(160))
print("Size of naiveFrequency: ", len(naiveFrequency))
print("Example: the = ", naiveFrequency['the'])
print(naiveFrequency_neg.most_common(160))
print("Size of naiveFrequency_neg: ", len(naiveFrequency_neg))
print("Example: the = ", naiveFrequency_neg['the'])
print(naiveFrequency_pos.most_common(160))
print("Size of naiveFrequency_pos:", len(naiveFrequency_pos))
print("Example: the = ", naiveFrequency_pos['the'])
'''
# print(naiveFrequency['all'].most_common(160))

'\nprint(naiveFrequency.most_common(160))\nprint("Size of naiveFrequency: ", len(naiveFrequency))\nprint("Example: the = ", naiveFrequency[\'the\'])\nprint(naiveFrequency_neg.most_common(160))\nprint("Size of naiveFrequency_neg: ", len(naiveFrequency_neg))\nprint("Example: the = ", naiveFrequency_neg[\'the\'])\nprint(naiveFrequency_pos.most_common(160))\nprint("Size of naiveFrequency_pos:", len(naiveFrequency_pos))\nprint("Example: the = ", naiveFrequency_pos[\'the\'])\n'

### Task 0.3 - Word Frequency without Stopword

In [29]:
# @author Pengnan Fan
# @acknowledgement Yuxiang Ma, for this function is edited based on his in miniproject1
# @param dataSet: set of comments
# @return naiveCount: word frequency without stopwords
def wordsFrequencyStopword(dataSet):
    stopwordCount = dict()
    totalString = str()
    count = 0
    lenW = len(dataSet['pos'])
    
    print("Start counting naive word frequency of positive set")
    
    for comment in dataSet['pos']:
        totalString = totalString + ' ' + comment['comment'].lower()
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    tokenizer = RegexpTokenizer(r'\w+')
    withoutPunc = tokenizer.tokenize(totalString)
    stopwordsSet = set(stopwords.words())
    posCountDict = Counter(s.lower() for s in withoutPunc if s.lower() not in stopwordsSet)
    print("\nComplete counting naive word frequency of positive set")
    
    totalString = str()
    count = 0
    lenW = len(dataSet['neg'])
    
    print("Start counting naive word frequency of negative set")
    
    for comment in dataSet['neg']:
        totalString = totalString + ' ' + comment['comment'].lower()
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    tokenizer = RegexpTokenizer(r'\w+')
    withoutPunc = tokenizer.tokenize(totalString)
    stopwordsSet = set(stopwords.words())
    negCountDict = Counter(s.lower() for s in withoutPunc if s.lower() not in stopwordsSet)
    print("\nComplete counting naive word frequency of negative set")
    
    totalString = str()
    count = 0
    lenW = len(dataSet['all'])
    
    print("Start counting naive word frequency of all set")
    
    for comment in dataSet['all']:
        totalString = totalString + ' ' + comment['comment'].lower()
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    tokenizer = RegexpTokenizer(r'\w+')
    withoutPunc = tokenizer.tokenize(totalString)
    stopwordsSet = set(stopwords.words())
    allCountDict = Counter(s.lower() for s in withoutPunc if s.lower() not in stopwordsSet)
    print("\nComplete counting naive word frequency of all set")
    
    stopwordCount = {'pos':posCountDict, 'neg':negCountDict, 'all':allCountDict}
    
    return stopwordCount

# stopwordFrequency = wordsFrequencyStopword(trainSet)

In [30]:
# An example of word frequency without stopwords
'''
print(stopwordFrequency.most_common(160))
print("Size of stopwordFrequency: ", len(stopwordFrequency))
print("Example: br = ", stopwordFrequency['br'])
print(stopwordFrequency_neg.most_common(160))
print("Size of stopwordFrequency_neg: ", len(stopwordFrequency_neg))
print("Example: movie = ", stopwordFrequency_neg['movie'])
print(stopwordFrequency_pos.most_common(160))
print("Size of stopwordFrequency_pos: ", len(stopwordFrequency_pos))
print("Example: movie = ", stopwordFrequency_pos['film'])
'''
# print(stopwordFrequency['all'].most_common(160))

'\nprint(stopwordFrequency.most_common(160))\nprint("Size of stopwordFrequency: ", len(stopwordFrequency))\nprint("Example: br = ", stopwordFrequency[\'br\'])\nprint(stopwordFrequency_neg.most_common(160))\nprint("Size of stopwordFrequency_neg: ", len(stopwordFrequency_neg))\nprint("Example: movie = ", stopwordFrequency_neg[\'movie\'])\nprint(stopwordFrequency_pos.most_common(160))\nprint("Size of stopwordFrequency_pos: ", len(stopwordFrequency_pos))\nprint("Example: movie = ", stopwordFrequency_pos[\'film\'])\n'

### Task 0.4 - Number of Naive Existance of Words

In [31]:
# @author Pengnan Fan
# @param dataSet: set of comments
# @return naiveCount: list of words of num of naive existances
def numOfExistanceNaive(dataSet):
    naiveCount = dict()
    totalComments = []
    count = 0
    lenW = len(dataSet['pos'])
    
    print("Start counting number of naive word existance of positive set")
    for comment in dataSet['pos']:
        commentSplit = comment['comment'].lower().split(" ")
        wordsToAdd = []
        for word in commentSplit:
            if word not in wordsToAdd:
                wordsToAdd.append(word)
        totalComments+=wordsToAdd
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    posCount = Counter(x for x in totalComments)
    print("\nComplete counting number of naive word existance of positive set")
    
    totalComments = []
    count = 0
    lenW = len(dataSet['neg'])
    
    print("Start counting number of naive word existance of negative set")
    for comment in dataSet['neg']:
        commentSplit = comment['comment'].lower().split(" ")
        wordsToAdd = []
        for word in commentSplit:
            if word not in wordsToAdd:
                wordsToAdd.append(word)
        totalComments+=wordsToAdd
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    negCount = Counter(x for x in totalComments)
    print("\nComplete counting number of naive word existance of negative set")
    
    totalComments = []
    count = 0
    lenW = len(dataSet['all'])
    
    print("Start counting number of naive word existance of all set")
    for comment in dataSet['all']:
        commentSplit = comment['comment'].lower().split(" ")
        wordsToAdd = []
        for word in commentSplit:
            if word not in wordsToAdd:
                wordsToAdd.append(word)
        totalComments+=wordsToAdd
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    allCount = Counter(x for x in totalComments)
    print("\nComplete counting number of naive word existance of all set")
    
    naiveCount = {'pos':posCount, 'neg':negCount, 'all':allCount}
    
    return naiveCount

# naiveNumOfExistance = numOfExistanceNaive(trainSet)

In [32]:
# An example of number of naive existance of words
'''
print(naiveNumOfExistance.most_common(160))
print("Size of naiveNumOfExistance: ", len(naiveNumOfExistance))
print("Example: the = ", naiveNumOfExistance['the'])
print(naiveNumOfExistance_neg.most_common(160))
print("Size of naiveNumOfExistance_neg: ", len(naiveNumOfExistance_neg))
print("Example: the = ", naiveNumOfExistance_neg['the'])
print(naiveNumOfExistance_pos.most_common(160))
print("Size of naiveNumOfExistance_pos: ", len(naiveNumOfExistance_pos))
print("Example: the = ", naiveNumOfExistance_pos['the'])
'''
# print(naiveNumOfExistance['pos'].most_common(160))

'\nprint(naiveNumOfExistance.most_common(160))\nprint("Size of naiveNumOfExistance: ", len(naiveNumOfExistance))\nprint("Example: the = ", naiveNumOfExistance[\'the\'])\nprint(naiveNumOfExistance_neg.most_common(160))\nprint("Size of naiveNumOfExistance_neg: ", len(naiveNumOfExistance_neg))\nprint("Example: the = ", naiveNumOfExistance_neg[\'the\'])\nprint(naiveNumOfExistance_pos.most_common(160))\nprint("Size of naiveNumOfExistance_pos: ", len(naiveNumOfExistance_pos))\nprint("Example: the = ", naiveNumOfExistance_pos[\'the\'])\n'

### Task 0.5 - Number of Existance of Words without Stopwords

In [33]:
# @author Pengnan Fan
# @param dataSet: set of comments
# @return naiveCount: list of num of existances of words without stopwords
def advancedNumOfExistance(dataSet):
    countDict = dict()
    stopwordsSet = set(stopwords.words())
    stopwordsSet.add('br')
    
    wordSet = list()
    count = 0
    lenW = len(dataSet['pos'])
    
    print("Start counting number of words without stopwords of existance of positive set")
    for exp in dataSet['pos']: 
        comment = contractions.fix(exp['comment'].lower())
        sent_map = comment.maketrans(dict.fromkeys(string.punctuation))
        sent_clean = comment.translate(sent_map)
        processedComment = ([k for k, v in groupby(sent_clean.split())])
        wordSet+=processedComment
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    posDict = Counter(s for s in wordSet if s not in stopwordsSet)
    
    print("\nComplete counting number of words without stopwords of existance of positive set")
    
    wordSet = list()
    count = 0
    lenW = len(dataSet['neg'])
    
    print("Start counting number of words without stopwords of existance of negative set")
    for exp in dataSet['neg']: 
        comment = contractions.fix(exp['comment'].lower())
        sent_map = comment.maketrans(dict.fromkeys(string.punctuation))
        sent_clean = comment.translate(sent_map)
        processedComment = ([k for k, v in groupby(sent_clean.split())])
        wordSet+=processedComment
            
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    negDict = Counter(s for s in wordSet if s not in stopwordsSet)
    
    print("\nComplete counting number of words without stopwords of existance of negative set")
    
    wordSet = list()
    count = 0
    lenW = len(dataSet['all'])
    
    print("Start counting number of words without stopwords of existance of all set")
    for exp in dataSet['all']: 
        comment = contractions.fix(exp['comment'].lower())
        sent_map = comment.maketrans(dict.fromkeys(string.punctuation))
        sent_clean = comment.translate(sent_map)
        processedComment = ([k for k, v in groupby(sent_clean.split())])
        wordSet+=processedComment
            
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1     
    
    allDict = Counter(s for s in wordSet if s not in stopwordsSet)
    
    print("\nComplete counting number of words without stopwords of existance of all set")
    
    countDict = {'neg':negDict, 'pos':posDict, 'all':allDict}
    
    return countDict


advancedNumOfExistance = advancedNumOfExistance(trainSet)

Start counting number of words without stopwords of existance of positive set
[=================== ] 99%
Complete counting number of words without stopwords of existance of positive set
Start counting number of words without stopwords of existance of negative set
[=================== ] 99%
Complete counting number of words without stopwords of existance of negative set
Start counting number of words without stopwords of existance of all set
[=================== ] 99%
Complete counting number of words without stopwords of existance of all set


In [34]:
# An example of number of existance of words without stopwords
'''
print(stopwordNumOfExistance.most_common(160))
print("Size of stopwordNumOfExistance: ", len(stopwordNumOfExistance))
print("Example: br = ", stopwordNumOfExistance['br'])
print(naiveNumOfExistance_neg.most_common(160))
print("Size of stopwordNumOfExistance_neg: ", len(stopwordNumOfExistance_neg))
print("Example: movie = ", stopwordNumOfExistance_neg['movie'])
print(naiveNumOfExistance_pos.most_common(160))
print("Size of stopwordNumOfExistance_pos: ", len(stopwordNumOfExistance_pos))
print("Example: movie = ", stopwordNumOfExistance_pos['film'])
'''
print(advancedNumOfExistance['neg'].most_common(160)) 
print(advancedNumOfExistance['all'].most_common(160))

[('movie', 23666), ('film', 17863), ('one', 12430), ('like', 10909), ('would', 9077), ('even', 7605), ('good', 7157), ('bad', 7003), ('really', 6074), ('time', 5779), ('could', 5703), ('see', 5378), ('get', 4981), ('story', 4969), ('much', 4928), ('people', 4636), ('make', 4548), ('made', 4342), ('first', 4237), ('movies', 4131), ('plot', 3964), ('acting', 3932), ('way', 3785), ('well', 3731), ('characters', 3687), ('think', 3612), ('films', 3462), ('watch', 3460), ('know', 3267), ('character', 3249), ('better', 3232), ('never', 3223), ('seen', 3180), ('ever', 3152), ('little', 3068), ('two', 3042), ('say', 2947), ('nothing', 2926), ('many', 2841), ('something', 2825), ('cannot', 2816), ('thing', 2783), ('show', 2761), ('scenes', 2708), ('scene', 2705), ('go', 2616), ('great', 2597), ('watching', 2591), ('going', 2438), ('worst', 2436), ('actually', 2431), ('actors', 2391), ('back', 2314), ('still', 2255), ('another', 2226), ('look', 2198), ('funny', 2196), ('us', 2192), ('life', 2177)

### Task 0.6 - Data Standardization

In [98]:
# @author Pengnan Fan
# @param dataSet
# @return proceedData
def dataStandardization(dataSet):
    proceedData = list()
    stopwordsSet = set(stopwords.words())
    stopwordsSet.add('br')
    
    count = 0
    lenW = len(dataSet)
    
    print("Start proceeding data")
    for data in dataSet: 
        comment = contractions.fix(data['comment'].lower())
        sent_map = comment.maketrans(dict.fromkeys(string.punctuation))
        sent_clean = comment.translate(sent_map)
        processedComment = sent_clean.split()
        toAdd = list()
        for x in processedComment:
            if x not in stopwordsSet and x not in toAdd:
                toAdd.append(x)
                
        proceedData.append(toAdd)
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    print("\nComplete proceeding data")
    return proceedData

proceedData_All = dataStandardization(trainSet['all'])

Start proceeding data
[=================== ] 99%
Complete proceeding data


## Task 1 - Bernoulli Naive Bayes

### Task 1.1 - Bernoulli Naive Bayes (Bugs here)

In [99]:
# @author Pengnan Fan
# @param dataSet: set for prediction
# @param wordExistance: dict {'pos' -> pos word existance, 'neg' -> neg word existance, 'all' -> all word existance}
# @param size: dict {'pos' -> pos size, 'neg' -> neg size, 'all' -> all size}
# @return prediction
def bernoulliNaiveBayes(dataSet, wordExistance, size):
    pPos = size['pos']/size['all']
    pNeg = size['neg']/size['all']
    prediction = list()
    count = 0
    lenW = len(dataSet)
    print("Start Bernoulli naive Bayes classifying")
    for exp in dataSet:
        pPos_x = pPos
        pNeg_x = pNeg
        
        for word in exp:
            pPos_x *= (wordExistance['pos'][word]+1)/(wordExistance['all'][word]+2)
            pNeg_x *= (wordExistance['neg'][word]+1)/(wordExistance['all'][word]+2)
        
        log_ratio = numpy.log([pPos_x/pNeg_x])
        
        if log_ratio > 0:
            prediction.append(1)
        else:
            prediction.append(0)
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    print("\nComplete Bernoulli naive Bayes classifying")
    
    return prediction

size = dict()
size = {'pos':len(trainSet['pos']), 'neg':len(trainSet['neg']), 'all':len(trainSet['all'])}
pred = bernoulliNaiveBayes(proceedData_All, advancedNumOfExistance, size)

Start Bernoulli naive Bayes classifying
[=================== ] 99%
Complete Bernoulli naive Bayes classifying


### Task 1.2 - Evaluation

In [100]:
# @author Pengnan Fan
# @param dataSet: train set with label
# @param prediction: prediction of each example in the trainSet
# @return result: list of results: {TP: true pos, TN: true neg, FP: false pos, FN: false neg}
def evaluation(dataSet, prediction):
    size = len(prediction)
    count = 0
    result = {'TP':0,'TN':0,'FP':0,'FN':0}
    
    print("Start evaluating classification")
    for i in range(size):
        if prediction[i]==1:
            if dataSet[i]['isPos']==1:
                result['TP']+=1
            else:
                result['FP']+=1
        else:
            if dataSet[i]['isPos']==1:
                result['FN']+=1
            else:
                result['TN']+=1
                
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(size)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    print("\nComplete evaluating classification")
    return result

eva = evaluation(trainSet['all'], pred)
print("TP = ", eva['TP'], "\nTN = ", eva['TN'], "\nFP = ", eva['FP'], "\nFN = ", eva['FN'])

Start evaluating classification
[=================== ] 99%
Complete evaluating classification
TP =  11848 
TN =  11482 
FP =  1018 
FN =  652


In [103]:
print("accuracy = ", (eva['TP']+eva['TN'])/250, "%")

accuracy =  93.32 %
